In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# import images as grayscale (pass cv2.IMREAD_GRAYSCALE or 0)
img_woman = cv2.imread('../data/Nadia_Murad.jpg', cv2.IMREAD_GRAYSCALE)
img_man = cv2.imread('../data/Denis_Mukwege.jpg', 0)
img_people = cv2.imread('../data/solvay_conference.jpg', 0)

In [ ]:
# returns None if loading image failed (for e.g. wrong path)
print(img_woman)

plt.imshow(img_woman)

In [ ]:
plt.imshow(img_woman, cmap='gray')

In [ ]:
plt.imshow(img_man, cmap='gray')

In [ ]:
plt.imshow(img_people, cmap='gray')

In [ ]:
# We need to create a classifier and pass in the XML classifier.
# OpenCV comes with pretrained cascade files.

# This is a list of ~6000 features that are going to be passed through the image to see if it fits all the features; 
# if it does, that means that the image contains a face.
face_classifier = cv2.CascadeClassifier('../data/haarcascades/haarcascade_frontalface_default.xml')

In [ ]:
def detect_faces(img):
    img_out = img.copy()
    rectangles = face_classifier.detectMultiScale(img_out)
    
    # (x, y) is top left corner
    for (x, y, w, h) in rectangles:
        # (255, 255, 255) is white color of the rectangle; 10 is thickness
        cv2.rectangle(img_out, (x, y), (x + w, y + h), (255, 255, 255), 10)
    return img_out

In [ ]:
result = detect_faces(img_man)

In [ ]:
plt.imshow(result, cmap='gray')

In [ ]:
result = detect_faces(img_woman)
plt.imshow(result, cmap='gray')

In [ ]:
result = detect_faces(img_people)
plt.imshow(result, cmap='gray')

In [ ]:
# As we can see above, it detects a gargoil as face and some faces are detected twice.
# To fix this, we'll use detectMultiScale's two additional parameters:
# - Scale factor: specifies how much is the image size reduced at each image scale
# - Minimum neighbours: how many neighbours each candidate rectangle should have to retain it. 
#                       Multiple rectangles detect the face, if they are near the same area or have a minimum number of neighbours that's where the face is.

def detect_faces_improved(img):
    img_out = img.copy()
    rectangles = face_classifier.detectMultiScale(img_out, scaleFactor = 1.2, minNeighbors = 5)
    
    # (x, y) is top left corner
    for (x, y, w, h) in rectangles:
        # (255, 255, 255) is white color of the rectangle; 10 is thickness
        cv2.rectangle(img_out, (x, y), (x + w, y + h), (255, 255, 255), 10)
    return img_out

In [ ]:
result = detect_faces_improved(img_people)
plt.imshow(result, cmap='gray')

In [ ]:
# As we can see in the outuput image, the face of the person who does not look at camera is not detected.
# This is because we're using Cascade classifier optimized/trained for detecting front-facing faces.

In [ ]:
eye_classifier = cv2.CascadeClassifier('../data/haarcascades/haarcascade_eye.xml')

In [ ]:
def detect_eyes_improved(img):
    img_out = img.copy()
    rectangles = eye_classifier.detectMultiScale(img_out, scaleFactor = 1.2, minNeighbors = 5)
    
    # (x, y) is top left corner
    for (x, y, w, h) in rectangles:
        # (255, 255, 255) is white color of the rectangle; 10 is thickness
        cv2.rectangle(img_out, (x, y), (x + w, y + h), (255, 255, 255), 10)
    return img_out

In [ ]:
result = detect_eyes_improved(img_woman)
plt.imshow(result, cmap='gray')

In [ ]:
result = detect_eyes_improved(img_man)
plt.imshow(result, cmap='gray')

In [ ]:
# As we can see from the output above, classifier didn't detect eyes on the man's face. 
# This is because within his eyes there is no enough contrast between dark eye centre (pupil) and white eye ball (sclera).

## Capturing faces/eyes on the video

In [ ]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read(0)
    frame = detect_faces_improved(frame)
    cv2.imshow('Video face detection', frame)
    
    key = cv2.waitKey(1)
    if key == 27:
        break
        
cap.release()
cv2.destroyAllWindows()